d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Managing Records

In the previous reading we demonstrated how to create a Delta table. We used basic data exploration strategies to identify two problems within the data. In this notebook, we will demonstrate how to correct those problems and write to a new, clean gold-level table that you can use for queries. Also, we will demonstrate how to repair and correct records

In this notebook, you will:
* Use a window function to interpolate missing values
* Update a Delta table
* Check the version history in a Delta table

## Getting started

Run the cell below to set up your classroom environment

In [0]:
%run ../Includes/8-2-Setup

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

res1: Boolean = true

res2: Boolean = false

res3: Boolean = false

-sandbox
## Repairing Records

In the previous reading, we found two problems in the data: 

1. We were missing records on a single device for a period of three days
1. There was at least one broken reading (heartrate less than zero) per day in our set

We will start by demonstrating how to merge a set of updates and insertions to repair these problems. 

First, we will work on the broken sensor readings. Previously, you used a window function, paired with the `AVG` function, to calculate an average value over a group of rows. Here, we will use a window function, paired with the built-in functions `LAG` and `LEAD`, to interpolate values to replace the broken readings. 

**`LAG`**: fetches data from the previous row. [Learn more](https://spark.apache.org/docs/latest/api/sql/#lag). <br>
**`LEAD`**: fetches data from a subsequent row. [Learn more](https://spark.apache.org/docs/latest/api/sql/#lead).<br>

Examine the code in the next cell. 

`line 1`: We create or replace a temporary view names `updates`<br>
`line 2`: We are using a CTAS pattern to create this new view<br>
`line 3`: We select a subgroup of columns to include from the window function defined in lines 5 - 8. Note the expression `(prev_amt+next_amt)/2`. For any missing entry, we calculate a new data point that is the mean of the previous entry and the subsequent entry. These values are defined in the window function below <br>
`line 4`: Indicates the window function as the source. The parenthesis marks the start of the window function<br>
`line 5`: Select all columns from `health_tracker_silver`<br>
`line 6`: `LAG` gets the `heartrate` from the previous row. We define the window by `device_id` and `dte` so that the calculation is applied to each missing value **by device** <br>
`line 9`: Marks the end of the window function<br>
`line 10`: Identifies that this calulation should apply **only** where the heartrate reading is less than 0

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> **Interpolation** is a type of estimation where we construct new data points based on a set of known data points.

In [0]:
%sql
SELECT *, 
    LAG(heartrate) OVER (PARTITION BY p_device_id, dte ORDER BY p_device_id, dte) AS prev_amt, 
    LEAD(heartrate) OVER (PARTITION BY p_device_id, dte ORDER BY p_device_id, dte) AS next_amt 
    FROM health_tracker_silver limit 5

name,heartrate,time,dte,p_device_id,prev_amt,next_amt
Deborah Powell,101.3720506847,2020-01-01T00:00:00.000+0000,2020-01-01,0,null,98.7361247811
Deborah Powell,98.7361247811,2020-01-01T01:00:00.000+0000,2020-01-01,0,101.3720506847,99.9724260825
Deborah Powell,99.9724260825,2020-01-01T02:00:00.000+0000,2020-01-01,0,98.7361247811,99.8718710286
Deborah Powell,99.8718710286,2020-01-01T03:00:00.000+0000,2020-01-01,0,99.9724260825,98.3444848503
Deborah Powell,98.3444848503,2020-01-01T04:00:00.000+0000,2020-01-01,0,99.8718710286,102.3191183486


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW updates 
AS (
  SELECT name, (prev_amt+next_amt)/2 AS heartrate, time, dte, p_device_id
  FROM (
    SELECT *, 
    LAG(heartrate) OVER (PARTITION BY p_device_id, dte ORDER BY p_device_id, dte) AS prev_amt, 
    LEAD(heartrate) OVER (PARTITION BY p_device_id, dte ORDER BY p_device_id, dte) AS next_amt 
    FROM health_tracker_silver
  ) 
  WHERE heartrate < 0 
)

## Check schema

We will want to use the values in `updates` to update our `health_tracker_silver` table. Let's check both schemas to see if they match.

In [0]:
%sql
DESCRIBE updates

col_name,data_type,comment
name,string,null
heartrate,double,null
time,timestamp,null
dte,date,null
p_device_id,bigint,null


In [0]:
%sql
DESCRIBE health_tracker_silver

col_name,data_type,comment
name,string,
heartrate,double,
time,timestamp,
dte,date,
p_device_id,bigint,
,,
# Partitioning,,
Part 0,p_device_id,


## Late-arriving data

We're ready to update our silver table with our interpolated values, but before we do, we find out that those missing readings have finally come through! We can get that data ready to merge with our other updates. 

Run the cell below to read in the raw data.

In [0]:
%sql
DROP TABLE IF EXISTS health_tracker_data_2020_02_late;              

CREATE TABLE health_tracker_data_2020_02_late                        
USING json                                             
OPTIONS (
  path "dbfs:/mnt/training/healthcare/tracker/raw-late.json",
  inferSchema "true"
  );

## Prepare inserts

We can apply the same transformations we used to create our `health_tracker_silver` table on this raw data. This we'll give us a view with the same schema as our other tables and views.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW inserts AS (
  SELECT
    value.name,
    value.heartrate,
    CAST(FROM_UNIXTIME(value.time) AS timestamp) AS time,
    CAST(FROM_UNIXTIME(value.time) AS DATE) AS dte,
    value.device_id p_device_id
  FROM
    health_tracker_data_2020_02_late
)

## Prepare upserts

The word "upsert" is a portmanteau of the words "update" and "insert," and this is what it does. An upsert will update records where some criteria are met and otherwise will insert the record. We create a view that is the union of our `updates` and `inserts` and holds all records we would like to add and modify.

Here, we use `UNION ALL` to capture all records in both views, even duplicates.

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW upserts
AS (
    SELECT * FROM updates 
    UNION ALL 
    SELECT * FROM inserts
    )

## Perform upsert 

When upserting into an existing Delta table, use Spark SQL to perform the merge from another registered table or view. The Transaction Log records the transaction, and the Metastore immediately reflects the changes.  

The merge appends both the new/inserted files and the files containing the updates to the Delta file directory. The transaction log tells the Delta reader which file to use for each record.

This operation is similar to the SQL `MERGE` command but has added support for deletes and other conditions in updates, inserts, and deletes. In other words, using the Spark SQL command `MERGE` provides full support for an upsert operation.

Use the comments to better understand how this command integrates records from our existing tables and views. 

Read more about `MERGE INTO` [here](https://docs.databricks.com/spark/latest/spark-sql/language-manual/merge-into.html#merge-into--delta-lake-on-databricks).

In [0]:
%sql
MERGE INTO health_tracker_silver                            -- the MERGE instruction is used to perform the upsert
USING upserts

ON health_tracker_silver.time = upserts.time AND        
   health_tracker_silver.p_device_id = upserts.p_device_id  -- ON is used to describe the MERGE condition
   
WHEN MATCHED THEN                                           -- WHEN MATCHED describes the update behavior
  UPDATE SET
  health_tracker_silver.heartrate = upserts.heartrate   
WHEN NOT MATCHED THEN                                       -- WHEN NOT MATCHED describes the insert behavior
  INSERT (name, heartrate, time, dte, p_device_id)              
  VALUES (name, heartrate, time, dte, p_device_id)

## Time travel

Let's check the number of records in the different versions of our tables. 

Version 1 shows the data after we added records from February. Recall that this is where we first discovered the missing records. 

The current version shows everything including the records we upcerted.

In [0]:
%sql
-- VERSION 1
SELECT COUNT(*) FROM health_tracker_silver VERSION AS OF 1

count(1)
7128


In [0]:
%sql
-- CURRENT VERSION
SELECT COUNT(*) FROM health_tracker_silver

count(1)
7200


## Describe history
 You can check the full history of a Delta table including the operation, user, and so on for each new write to a table.

In [0]:
%sql
DESCRIBE HISTORY health_tracker_silver

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2020-12-11T17:02:53.000+0000,7473171008449620,galinablokh@gmail.com,MERGE,"Map(predicate -> ((spark_catalog.default.health_tracker_silver.`time` = upserts.`time`) AND (spark_catalog.default.health_tracker_silver.`p_device_id` = upserts.`p_device_id`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3431324887126229),1211-145209-tibia991,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 7054, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 5, numTargetRowsInserted -> 72, numTargetRowsUpdated -> 74, numOutputRows -> 7200, numSourceRows -> 146, numTargetFilesRemoved -> 10)",null
1,2020-12-11T15:47:01.000+0000,7473171008449620,galinablokh@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3431324887126229),1211-145209-tibia991,0,WriteSerializable,true,"Map(numFiles -> 5, numOutputBytes -> 52875, numOutputRows -> 3408)",null
0,2020-12-11T15:46:41.000+0000,7473171008449620,galinablokh@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> false, description -> null, partitionBy -> [""p_device_id""], properties -> {})",null,List(3431324887126229),1211-145209-tibia991,null,WriteSerializable,false,"Map(numFiles -> 5, numOutputBytes -> 57098, numOutputRows -> 3720)",null


## Write to gold

So far, we have ingested raw (bronze-level) data and applied transformations to create a silver table. We have used Spark SQL to explore and transform that data further, adding new values when we found collection errors and updating the table to reflect late-arriving data. Now that our data is clean and polished, we can write to a gold table. Gold tables are used to hold business level aggregates. When we create this table, we also apply aggregate functions to several columns.

In [0]:
%sql
DROP TABLE IF EXISTS health_tracker_gold;              

CREATE TABLE health_tracker_gold                        
USING DELTA
LOCATION "/health_tracker/gold"
AS 
SELECT 
  AVG(heartrate) AS meanHeartrate,
  STD(heartrate) AS stdHeartrate,
  MAX(heartrate) AS maxHeartrate
FROM health_tracker_silver
GROUP BY p_device_id


In [0]:
%sql
SELECT
  *
FROM
  health_tracker_gold

meanHeartrate,stdHeartrate,maxHeartrate
87.56374275065477,28.697966782294422,191.7364805027
82.8938339413805,24.627453529979476,174.3611679317
82.49632786583678,22.460895905499,192.1828472326
86.40550075283315,25.919716960302623,177.6570899987
80.99397568360457,23.832859676185365,175.8194964652


## Cleanup
Run the next cell to clean up your classroom environment

In [0]:
%run ../Includes/Classroom-Cleanup

Great work! Now that you've got a basic understanding of how data moves through the Delta architecture, we're ready to get back to analytics. In the next reading, we'll see how to write high-performance Spark queries with Databricks Delta.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>